<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/ensembling_DNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensembling DNN, Random Forest and DecisionTree based on Normalization and Standardization Datasets


We will use the same DNN model, Random Forest and Decision Tree to classify network anormalies in CICIDS2017 dataset. More specifically, the dataset has been augmented by adding 7 new connection based features. We will use normalized and standardized datasets to train the DNN model. Random Foresty model is not sensitive to the normalization method. Then each model will be evaluated using the test set with the corresponding normalized dataset. After that, we put all two models in an ensemble and evaluate it. It is expected that the ensemble will perform better on a test set that any single model in the ensemble separately.

There are many different types of ensembles; stacking is one of them. It is one of the more general types and can theoretically represent any other ensemble technique. Stacking involves training a learning algorithm to combine the predictions of several other learning algorithms. For the sake of this example, I will use one of the simplest forms of Stacking, which involves taking an average of outputs of models in the ensemble. Since averaging doesn't take any parameters, there is no need to train this ensemble (only its models).

## Preparing the data
First, import dependencies.

In [ ]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os
from os.path import join
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt


### Define Metrics

In [ ]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [ ]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [ ]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

# 1. Locating CSV files

In [ ]:
# All columns
col_names = np.array(['dst sport count', 'src dport count', 'dst src count', 'dport count', 'sport count', 'dst host count','src host count','Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

### Option 1. Connect to Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path='/content/drive/My Drive/CICIDS2017/train_set_ext78_2.csv'

In [ ]:
validation_path = '/content/drive/My Drive/CICIDS2017/crossval_set_ext78_2.csv'
test_path = '/content/drive/My Drive/CICIDS2017/test_set_ext78_2.csv'

### Option 2. Connect to Local Machine

In [ ]:
train_path = '../data/cicids2017clean/train_set_ext78_2.csv'
validation_path = '../data/cicids2017clean/crossval_set_ext78_2.csv'
test_path = '../data/cicids2017clean/test_set_ext78_2.csv'

# 2. Loading CSV Datasets

In [ ]:
# load three csv files generated by mlp4nids (Multi-layer perceptron for network intrusion detection )
# first load the train set
df_train = pd.read_csv(train_path,names=col_names, skiprows=1)  

In [ ]:
print('Train set size: ', df_train.shape)

Train set size:  (879589, 79)


In [ ]:
df_test = pd.read_csv(test_path, names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)

df_val = pd.read_csv(validation_path,names=col_names, skiprows=1)  
print('Validation set size: ', df_val.shape)

Test set size:  (188483, 79)
Validation set size:  (188484, 79)


In [ ]:
df_train.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,879589.000000,8.795890e+05,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,879589.000000,8.795890e+05,879589.000000,8.795890e+05,879589.000000,879589.000000,879589.000000,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05,8.795890e+05
mean,9.655316,37.867227,49.836973,48.006394,10.465226,55.566170,57.243626,42882.307181,6397.910982,8.693028,1.923037e+07,8.214027,8.823871,4.403727e+02,1.417082e+04,179.036862,13.701055,48.020170,61.747571,1463.559232,29.061780,487.495944,600.398140,1.154200e+06,7.835547e+04,1.785500e+06,4.593409e+06,1.505781e+07,1.817909e+05,1.895005e+07,3.497445e+06,5.784826e+06,1.493529e+07,8.764154e+05,9.546793e+06,1.832198e+06,2.096462e+06,5.847373e+06,7.775011e+05,0.035845,0.000069,-1.779193e+03,-1.391305e+03,7.066763e+04,7.731049e+03,11.697604,1520.444635,250.439207,478.990505,9.488540e+05,0.056930,0.035845,0.000164,0.348102,0.351718,0.068783,0.000069,0.000164,0.662657,277.045229,48.020170,487.495944,8.214027,4.403727e+02,8.823871,1.416974e+04,7118.105536,1463.521291,4.322587,-1.156058e+03,8.722149e+04,3.427464e+04,1.409675e+05,6.695474e+04,1.402000e+07,8.088585e+05,1.463478e+07,1.341685e+07
std,25.810487,40.954725,44.142062,39.181947,25.970811,41.115493,40.505493,19661.359246,16111.253716,4.734203,3.711919e+07,685.761027,912.882257,6.605092e+03,2.075330e+06,610.645248,57.039193,160.224075,237.180304,2657.258586,60.386127,814.918490,1167.665597,2.334998e+07,2.713128e+05,4.929222e+06,9.877418e+06,3.194961e+07,3.274780e+06,3.708712e+07,9.762596e+06,1.305067e+07,3.201589e+07,8.098459e+06,2.816566e+07,8.481475e+06,8.133173e+06,2.054948e+07,7.499528e+06,0.185904,0.008327,1.186150e+06,1.158914e+06,2.652601e+05,3.809361e+04,22.706568,2687.748532,394.185794,848.222439,2.302125e+06,0.231709,0.185904,0.012794,0.476369,0.477507,0.253085,0.008327,0.012794,0.643625,431.918903,160.224075,814.918490,685.761027,6.605092e+03,912.882257,2.075043e+06,13584.239160,7163.856980,

In [ ]:
df_test.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,1.884830e+05,188483.000000,188483.000000,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05
mean,9.625308,37.861701,49.864025,48.026984,10.434315,55.607434,57.249322,42825.772430,6398.028565,8.699867,1.937297e+07,6.393341,6.290955,425.507547,9.614426e+03,180.326422,13.616549,48.088141,62.258627,1471.358101,29.252240,489.581182,604.521501,1.141231e+06,7.836992e+04,1.791253e+06,4.628396e+06,1.515027e+07,1.736502e+05,1.910116e+07,3.521722e+06,5.821567e+06,1.502940e+07,8.824922e+05,9.685500e+06,1.863613e+06,2.133658e+06,5.939107e+06,7.890398e+05,0.036115,0.000095,-7.244541e+02,-7.290569e+02,7.076928e+04,7.712062e+03,11.748036,1528.642275,251.232699,481.683733,9.575668e+05,0.057278,0.036115,0.000164,0.348620,0.350392,0.068335,0.000095,0.000175,0.662680,277.964550,48.088141,489.581182,6.393341,425.507547,6.290955,9.615250e+03,7121.997130,1479.610888,3.475565,-4.185286e+02,8.921930e+04,3.533351e+04,1.448189e+05,6.831514e+04,1.412079e+07,8.001563e+05,1.473072e+07,1.352238e+07
std,25.759133,40.995800,44.175627,39.217990,25.916640,41.132627,40.537490,19694.807197,16123.799361,4.737968,3.724073e+07,364.106378,421.965728,3858.850242,1.454190e+06,618.937180,55.371692,158.738971,238.828812,2667.682650,62.105876,817.093491,1174.621985,2.348509e+07,2.727351e+05,4.872676e+06,9.915489e+06,3.202428e+07,3.155030e+06,3.721579e+07,9.783012e+06,1.309209e+07,3.209203e+07,8.108421e+06,2.836729e+07,8.531263e+06,8.212071e+06,2.070359e+07,7.526471e+06,0.186576,0.009772,3.865090e+05,3.865304e+05,2.665324e+05,3.733633e+04,23.000785,2699.393195,394.531874,851.827223,2.321479e+06,0.232374,0.186576,0.012824,0.476535,0.477094,0.252321,0.009772,0.013231,0.648263,432.349886,158.738971,817.093491,364.106378,3858.850242,421.965728,1.454292e+06,13565.260196,7247.906478,355.

In [ ]:
df_val.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,188484.000000,1.884840e+05,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,188484.000000,1.884840e+05,188484.000000,1.884840e+05,188484.000000,188484.000000,188484.000000,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05,1.884840e+05
mean,9.731272,37.780172,49.883332,47.914115,10.539032,55.614689,57.218512,42898.015243,6422.896426,8.694239,1.924685e+07,7.434695,7.835328,4.390308e+02,1.180121e+04,178.156406,13.800163,48.024119,61.523877,1463.444457,29.060005,487.327309,600.629474,1.225907e+06,7.942923e+04,1.791008e+06,4.609148e+06,1.513562e+07,1.849447e+05,1.896832e+07,3.499186e+06,5.813441e+06,1.501155e+07,8.688593e+05,9.470943e+06,1.831885e+06,2.100258e+06,5.840577e+06,7.745564e+05,0.035764,0.000069,-6.292069e+04,-1.164520e+04,7.168020e+04,7.802082e+03,11.720783,1520.258568,249.863420,478.533085,9.479145e+05,0.056583,0.035764,0.000154,0.347573,0.352502,0.068372,0.000069,0.000154,0.660905,276.454458,48.024119,487.327309,7.434695,4.390308e+02,7.835328,1.179981e+04,7106.119968,1484.061920,3.641153,-8.963623e+03,9.042821e+04,3.565553e+04,1.450080e+05,6.899322e+04,1.407187e+07,8.205580e+05,1.469736e+07,1.346096e+07
std,25.915515,40.960165,44.147226,39.203217,26.082113,41.112314,40.531929,19646.086134,16145.734341,4.735494,3.709354e+07,561.529560,747.556849,6.996634e+03,1.668861e+06,604.186341,57.791361,160.261305,237.467628,2660.075880,60.839255,816.166175,1169.171476,2.482471e+07,2.735719e+05,4.871805e+06,9.865068e+06,3.198730e+07,3.220118e+06,3.705975e+07,9.655176e+06,1.307159e+07,3.205087e+07,7.956019e+06,2.804916e+07,8.424538e+06,8.163563e+06,2.053212e+07,7.404418e+06,0.185702,0.008305,2.253288e+07,3.503013e+06,2.672851e+05,3.867133e+04,22.334888,2689.274156,393.766139,847.928470,2.293114e+06,0.231045,0.185702,0.012403,0.476201,0.477751,0.252384,0.008305,0.012403,0.644010,431.481560,160.261305,816.166175,561.529560,6.996634e+03,747.556849,1.668515e+06,13575.776949,7232.426279,

# 3. Encoding Datasets

### Encoding train dataset

In [ ]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
Xtrain = data.values
y_train = encode_label(df_label.values)

### Encoding test dataset

In [ ]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

### Encoding validation dataset

In [ ]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
Xval = data.values
y_val = encode_label(df_label.values)

# 4. Normalization

The values of the datasets are normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train_n = scaler.fit_transform(Xtrain)
X_train_n

array([[0.01010101, 1.        , 1.        , ..., 0.68650794, 0.71416667,
        0.10166667],
       [0.01010101, 0.22222222, 0.12121212, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01010101, 0.02020202, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.12121212, 0.12121212, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.87878788, 0.87878788, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
X_val_n = scaler.fit_transform(Xval)
X_val_n

array([[0.        , 0.32323232, 0.32323232, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05050505, 0.05050505, ..., 0.        , 0.        ,
        0.        ],
       [0.02020202, 0.09090909, 0.02020202, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.96969697, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
X_test_n = scaler.fit_transform(Xtest)
X_test_n

array([[0.        , 0.96969697, 0.96969697, ..., 0.        , 0.71583333,
        0.71583333],
       [0.        , 0.96969697, 0.96969697, ..., 0.        , 0.6975    ,
        0.6975    ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.44444444, 0.08080808, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.78787879, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.28282828, 0.28282828, ..., 0.        , 0.        ,
        0.        ]])

# 5. Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train_sd = scaler.fit_transform(Xtrain)
X_val_sd = scaler.fit_transform(Xval)
X_test_sd = scaler.fit_transform(Xtest)

X_train_sd

array([[-0.29659729,  1.51710965,  1.13640038, ...,  8.4581104 ,
         2.22140524, -0.0393603 ],
       [-0.29659729, -0.36301636, -0.83451004, ..., -0.13390608,
        -0.45746381, -0.43398332],
       [-0.33534125, -0.87577799, -1.06105146, ..., -0.13390608,
        -0.45746381, -0.43398332],
       ...,
       [-0.33534125, -0.60718856, -0.83451004, ..., -0.13390608,
        -0.45746381, -0.43398332],
       [-0.33534125,  1.224103  ,  0.86455067, ..., -0.13390608,
        -0.45746381, -0.43398332],
       [ 3.50031109, -0.90019522,  1.13640038, ..., -0.13390608,
        -0.45746381, -0.43398332]])

# 6. One-hot Encoding for labels

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train_origin = y_train
y_test_origin = y_test
y_val_origin = y_val

In [ ]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

# 7.  Define the Metrics

In [ ]:
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
labels_d = make_value2index(df_test['Label'])

In [ ]:
print(labels_d)

{'BENIGN': 105018, 'Bot': 105298, 'DDoS': 124569, 'DoS GoldenEye': 126111, 'DoS Hulk': 160658, 'DoS Slowhttptest': 161486, 'DoS slowloris': 162320, 'FTP-Patator': 163498, 'Heartbleed': 163500, 'Infiltration': 163501, 'PortScan': 187347, 'SSH-Patator': 188173, 'Web Attack � Brute Force': 188382, 'Web Attack � Sql Injection': 188389, 'Web Attack � XSS': 188482}


# First model:  Random Foresty with DecisionTree

### The first model is Random Foresty with DecisionTree.  

In [ ]:
randomforest = RandomForestClassifier(n_estimators=10, random_state=10)
randomforest.fit(X_train_n,y_train)
    
y_pred = randomforest.predict(X_test_n)

In [ ]:
display_metrics(y_test_origin, np.argmax(y_pred, axis = 1), labels_d)


Accuracy: 0.92

Micro Precision: 0.92
Micro Recall: 0.92
Micro F1-score: 0.92

Macro Precision: 0.91
Macro Recall: 0.75


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.80

Weighted Precision: 0.93
Weighted Recall: 0.92
Weighted F1-score: 0.91

Classification Report



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       0.88      1.00      0.93    105019
                       Bot       0.95      0.19      0.31       280
                      DDoS       1.00      0.91      0.95     19271
             DoS GoldenEye       1.00      0.92      0.96      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      0.99      0.99       828
             DoS slowloris       1.00      0.90      0.95       834
               FTP-Patator       1.00      0.53      0.69      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
                  PortScan       1.00      0.51      0.68     23846
               SSH-Patator       1.00      0.99      1.00       826
  Web Attack � Brute Force       0.91      0.75      0.82       209
Web Attack � Sql Injection       0.00      0.00

### Save the model to disk

In [ ]:
import pickle

randomforest_file_name = 'randomforest.sav'
pickle.dump(randomforest, open(randomforest_file_name, 'wb'))

# Second Model: Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_dec = DecisionTreeClassifier()
model_dec.fit(X_train_n, y_train_origin)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred_tree = model_dec.predict(X_test_n)

In [ ]:
display_metrics(y_test_origin, y_pred_tree, labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.87
Macro Recall: 0.93
Macro F1-score: 0.88

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       0.99      0.99      0.99    105019
                       Bot       0.69      0.96      0.80       280
                      DDoS       1.00      0.97      0.99     19271
             DoS GoldenEye       0.68      0.80      0.74      1542
                  DoS Hulk       1.00      0.98      0.99     34547
          DoS Slowhttptest       0.95      0.98      0.97       828
             DoS slowloris       0.89      0.95      0.92       834
               FTP-Patator       1.00      0.99      0.99      1178
                Heartbleed       0.67      1.00      0.80         2
              Infiltration       0.50      1.00      0.67         1
             

### Save the model

In [ ]:
decisiontree_file_name = 'decisiontree.sav'
pickle.dump(model_dec, open(decisiontree_file_name, 'wb'))

# Third model: DNN Model

We will train the same DNN model by using the same dataset with different normalization methods.

In [ ]:
def make_model(X_train, y_train, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          256, activation='relu',
          input_shape=(X_train.shape[-1],)),
      tf.keras.layers.Dense(256, activation ='relu'),
      tf.keras.layers.Dense(128, activation ='relu'),
      tf.keras.layers.Dense(64, activation ='relu'),
      tf.keras.layers.Dense(y_train.shape[-1], activation='softmax',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(lr=1e-4),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=METRICS)
    
  return model

We use normalized dataset to train this DNN model.

In [ ]:
model_dnn_n = make_model(X_train_n, y_train)
model_dnn_sd = make_model(X_train_sd, y_train) 

### Train the first DNN model with normalized dataset

In [ ]:
EPOCHS = 200
BATCH_SIZE = 9500

In [ ]:
baseline_history_n = model_dnn_n.fit(
    X_train_n,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_n, y_val))

Epoch 1/100
93/93 [==============================] - 4s 39ms/step - loss: 0.0031 - tp: 872753.0000 - fp: 6710.0000 - tn: 12307536.0000 - fn: 6836.0000 - accuracy: 0.9990 - precision: 0.9924 - recall: 0.9922 - auc: 0.9991 - val_loss: 0.0037 - val_tp: 186722.0000 - val_fp: 1743.0000 - val_tn: 2637033.0000 - val_fn: 1762.0000 - val_accuracy: 0.9988 - val_precision: 0.9908 - val_recall: 0.9907 - val_auc: 0.9987
Epoch 2/100
93/93 [==============================] - 4s 38ms/step - loss: 0.0031 - tp: 872685.0000 - fp: 6797.0000 - tn: 12307449.0000 - fn: 6904.0000 - accuracy: 0.9990 - precision: 0.9923 - recall: 0.9922 - auc: 0.9990 - val_loss: 0.0036 - val_tp: 186805.0000 - val_fp: 1652.0000 - val_tn: 2637124.0000 - val_fn: 1679.0000 - val_accuracy: 0.9988 - val_precision: 0.9912 - val_recall: 0.9911 - val_auc: 0.9990
Epoch 3/100
93/93 [==============================] - 4s 38ms/step - loss: 0.0031 - tp: 872710.0000 - fp: 6776.0000 - tn: 12307470.0000 - fn: 6879.0000 - accuracy: 0.9990 - precis

In [ ]:
y_pred_n=model_dnn_n.predict(X_test_n)

In [ ]:
display_metrics(y_test_origin, np.argmax(y_pred_n, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.88
Macro Recall: 0.87


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.87

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      0.99      0.99    105019
                       Bot       0.86      0.99      0.92       280
                      DDoS       0.98      1.00      0.99     19271
             DoS GoldenEye       1.00      0.94      0.97      1542
                  DoS Hulk       1.00      0.99      0.99     34547
          DoS Slowhttptest       0.99      0.99      0.99       828
             DoS slowloris       0.98      0.97      0.98       834
               FTP-Patator       1.00      0.99      0.99      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
                  PortScan       0.96      0.99      0.98     23846
               SSH-Patator       0.98      0.98      0.98       826
  Web Attack � Brute Force       0.75      0.82      0.78       209
Web Attack � Sql Injection       0.00      0.00

### Save the model

In [ ]:
model_dnn_n.save('dnn_n.h5')

### Train the second DNN model with standardized dataset

In [ ]:
EPOCHS = 100
BATCH_SIZE = 9500

In [ ]:
baseline_history_sd = model_dnn_sd.fit(
    X_train_sd,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_sd, y_val))

Epoch 1/100
93/93 [==============================] - 4s 40ms/step - loss: 7.1730e-04 - tp: 878428.0000 - fp: 1129.0000 - tn: 12313117.0000 - fn: 1161.0000 - accuracy: 0.9998 - precision: 0.9987 - recall: 0.9987 - auc: 0.9999 - val_loss: 0.0019 - val_tp: 187966.0000 - val_fp: 512.0000 - val_tn: 2638264.0000 - val_fn: 518.0000 - val_accuracy: 0.9996 - val_precision: 0.9973 - val_recall: 0.9973 - val_auc: 0.9997
Epoch 2/100
93/93 [==============================] - 4s 38ms/step - loss: 7.0809e-04 - tp: 878437.0000 - fp: 1123.0000 - tn: 12313123.0000 - fn: 1152.0000 - accuracy: 0.9998 - precision: 0.9987 - recall: 0.9987 - auc: 0.9999 - val_loss: 0.0018 - val_tp: 188069.0000 - val_fp: 411.0000 - val_tn: 2638365.0000 - val_fn: 415.0000 - val_accuracy: 0.9997 - val_precision: 0.9978 - val_recall: 0.9978 - val_auc: 0.9998
Epoch 3/100
93/93 [==============================] - 4s 38ms/step - loss: 7.0017e-04 - tp: 878462.0000 - fp: 1102.0000 - tn: 12313144.0000 - fn: 1127.0000 - accuracy: 0.9998 

In [ ]:
y_pred_sd = model_dnn_sd.predict(X_test_sd)

In [ ]:
display_metrics(y_test_origin, np.argmax(y_pred_sd, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.73
Macro Recall: 0.69


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.70

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.98

Classification Report



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       0.99      0.98      0.99    105019
                       Bot       0.83      0.98      0.90       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       0.99      0.99      0.99      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      0.99      0.99       828
             DoS slowloris       0.99      0.99      0.99       834
               FTP-Patator       0.99      0.51      0.67      1178
                Heartbleed       0.00      0.00      0.00         2
              Infiltration       0.00      0.00      0.00         1
                  PortScan       0.94      1.00      0.97     23846
               SSH-Patator       0.90      0.48      0.63       826
  Web Attack � Brute Force       0.77      0.75      0.76       209
Web Attack � Sql Injection       0.00      0.00

### Save the trained model

In [ ]:
model_dnn_sd.save('dnn_sd.h5')

# Training DNN with mixed data normalization

## 1. Load the DNN model trained with normalized dataset

In [ ]:
from keras.models import load_model

In [ ]:
dnn_n_model = load_model('dnn_n.h5')

In [ ]:
dnn_n_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               20224     
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 15)                975       
Total params: 128,143
Trainable params: 128,143
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dnn_n_model# update all layers in all models to not be trainable

for layer in dnn_n_model.layers:
		# make trainable
    layer.trainable = True
    print(layer.name)

dense_10
dense_11
dense_12
dense_13
dense_14


In [ ]:
layer = dnn_n_model.get_layer('dense_10')
layer.trainable=True

layer = dnn_n_model.get_layer('dense_11')
layer.trainable=False

## 2. Training the model by using standardized dataset

In [ ]:
EPOCHS = 80
BATCH_SIZE = 9000

In [ ]:
history_sd = dnn_n_model.fit(
    X_train_sd,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_sd, y_val))

Epoch 1/80
98/98 [==============================] - 5s 45ms/step - loss: 1.2784 - tp: 750185.0000 - fp: 128661.0000 - tn: 12185585.0000 - fn: 129404.0000 - accuracy: 0.9804 - precision: 0.8536 - recall: 0.8529 - auc: 0.9271 - val_loss: 0.1674 - val_tp: 176590.0000 - val_fp: 11677.0000 - val_tn: 2627099.0000 - val_fn: 11894.0000 - val_accuracy: 0.9917 - val_precision: 0.9380 - val_recall: 0.9369 - val_auc: 0.9739
Epoch 2/80
98/98 [==============================] - 4s 37ms/step - loss: 0.1081 - tp: 834056.0000 - fp: 43524.0000 - tn: 12270722.0000 - fn: 45533.0000 - accuracy: 0.9932 - precision: 0.9504 - recall: 0.9482 - auc: 0.9794 - val_loss: 0.0707 - val_tp: 180320.0000 - val_fp: 7808.0000 - val_tn: 2630968.0000 - val_fn: 8164.0000 - val_accuracy: 0.9944 - val_precision: 0.9585 - val_recall: 0.9567 - val_auc: 0.9834
Epoch 3/80
98/98 [==============================] - 4s 36ms/step - loss: 0.0521 - tp: 845075.0000 - fp: 33190.0000 - tn: 12281056.0000 - fn: 34514.0000 - accuracy: 0.9949 -

In [ ]:
y_pred_mix = dnn_n_model.predict(X_test_sd)

In [ ]:
y_pred_mix

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.0891338e-09, 7.7628711e-12, 1.4513332e-22, ..., 3.4873085e-15,
        1.2335191e-19, 3.6827611e-12],
       ...,
       [1.0000000e+00, 1.0418901e-34, 0.0000000e+00, ..., 5.9388984e-32,
        1.4865620e-36, 0.0000000e+00],
       [1.1582623e-08, 5.4412169e-10, 1.2817214e-21, ..., 1.1954169e-12,
        8.6470095e-18, 6.4685195e-11],
       [1.0000000e+00, 3.2562312e-34, 2.7089362e-36, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [ ]:
display_metrics(y_test_origin, np.argmax(y_pred_mix, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.81
Macro Recall: 0.78


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.78

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      0.99      0.99    105019
                       Bot       0.85      0.90      0.87       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       0.99      0.98      0.99      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       0.99      0.98      0.99       828
             DoS slowloris       0.97      0.99      0.98       834
               FTP-Patator       0.99      0.99      0.99      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       0.00      0.00      0.00         1
                  PortScan       0.97      0.99      0.98     23846
               SSH-Patator       0.97      0.98      0.97       826
  Web Attack � Brute Force       0.65      0.78      0.71       209
Web Attack � Sql Injection       0.00      0.00

### Save the trained model

In [ ]:
dnn_n_model.save('dnn_mix.h5')

#  **Four Model Ensemble**

Now all three models will be combined in an ensemble. 

Here, all four models are reinstantiated and the best saved weights are loaded.

If we want to reload the models with saved weights, we reload the saved models

In [ ]:
results = np.zeros((y_test_origin.shape[0], 15))

In [ ]:
y_pred_tree_cat = to_categorical(y_pred_tree, 15)

In [ ]:
results = results+y_pred*10+y_pred_n+y_pred_sd+y_pred_mix+y_pred_tree_cat*10

In [ ]:
display_metrics(y_test_origin, np.argmax(results, axis = 1) , labels_d)


Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.98
Macro Recall: 0.92
Macro F1-score: 0.94

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    105019
                       Bot       0.98      0.96      0.97       280
                      DDoS       1.00      0.99      1.00     19271
             DoS GoldenEye       1.00      0.95      0.97      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      0.99      1.00       828
             DoS slowloris       0.96      0.99      0.98       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
             

# Pre-trained Models and Weights
If you do not want to train the models, you can use pre-trained models and their weights.

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
randomforest_file_name = 'randomforest.sav'
decisiontree_file_name = 'decisiontree.sav'

In [ ]:
# load the Random Forest model from disk
randomforest_model = pickle.load(open(randomforest_file_name, 'rb'))
decisiontree_model = pickle.load(open(decisiontree_file_name, 'rb'))
dnn_n_model = load_model('dnn_n.h5',compile = False)
dnn_sd_model = load_model('dnn_sd.h5',compile = False)
dnn_mix_model = load_model('dnn_mix.h5',compile = False) 

C:\Users\fwang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\fwang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Get the prediction values

In [ ]:
y_pred = randomforest_model.predict(X_test_n)

In [ ]:
y_pred_tree = decisiontree_model.predict(X_test_n)

In [ ]:
y_pred_dnn_n = dnn_n_model.predict(X_test_n)

In [ ]:
y_pred_dnn_sd = dnn_sd_model.predict(X_test_n)

In [ ]:
y_pred_dnn_mix = dnn_mix_model.predict(X_test_n)

In [ ]:
display_metrics(y_test_origin, np.argmax(y_pred_dnn_sd, axis = 1) , labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.88
Macro Recall: 0.87


C:\Users\fwang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1-score: 0.87

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report



C:\Users\fwang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      0.99      0.99    105019
                       Bot       0.86      0.99      0.92       280
                      DDoS       0.98      1.00      0.99     19271
             DoS GoldenEye       1.00      0.94      0.97      1542
                  DoS Hulk       1.00      0.99      0.99     34547
          DoS Slowhttptest       0.99      0.99      0.99       828
             DoS slowloris       0.98      0.97      0.98       834
               FTP-Patator       1.00      0.99      0.99      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
                  PortScan       0.96      0.99      0.98     23846
               SSH-Patator       0.98      0.98      0.98       826
  Web Attack � Brute Force       0.75      0.82      0.78       209
Web Attack � Sql Injection       0.00      0.00

### Define a filter for each model

In [ ]:
def filter(pred, list):
    for i in list:
        pred[:, i] = 0
    return pred

In [ ]:
random_list=[2,4,8,9,13]
tree_list = [2,3,4,6,8,9]
dnn_n_list = [2,4,13]
dnn_sd_list = [4,6,8,9,11,13]

In [ ]:
y_pred_tree_cat = to_categorical(y_pred_tree, 15)

In [ ]:
random_pred_filter=filter(y_pred, random_list)
tree_pred_filter=filter(y_pred_tree_cat, tree_list)
dnn_n_pred_filter=filter(y_pred_dnn_n, dnn_n_list)
dnn_sd_pred_filter=filter(y_pred_dnn_sd, dnn_sd_list)

In [ ]:
dnn_n_pred_filter

array([[2.25684058e-28, 1.23421603e-26, 0.00000000e+00, ...,
        5.93876203e-32, 0.00000000e+00, 1.30338280e-19],
       [3.07559069e-30, 2.64064061e-34, 0.00000000e+00, ...,
        2.03038143e-33, 0.00000000e+00, 9.98061425e-21],
       [6.42797315e-10, 4.76133356e-15, 0.00000000e+00, ...,
        3.35292570e-26, 0.00000000e+00, 1.11194267e-17],
       ...,
       [1.00000000e+00, 3.83469262e-23, 0.00000000e+00, ...,
        1.61605880e-33, 0.00000000e+00, 5.29033394e-24],
       [1.17106474e-07, 5.25993962e-12, 0.00000000e+00, ...,
        7.37398901e-23, 0.00000000e+00, 1.31422943e-15],
       [1.00000000e+00, 1.80899039e-24, 0.00000000e+00, ...,
        1.73941646e-32, 0.00000000e+00, 2.67943355e-26]], dtype=float32)

In [ ]:
results = np.zeros((y_test_origin.shape[0], 15))

results = results+y_pred*10 + y_pred_dnn_n + y_pred_dnn_sd + y_pred_dnn_mix + y_pred_tree_cat*10

In [ ]:
display_metrics(y_test_origin, np.argmax(results, axis = 1) , labels_d)


Accuracy: 0.81

Micro Precision: 0.81
Micro Recall: 0.81
Micro F1-score: 0.81

Macro Precision: 0.56
Macro Recall: 0.78
Macro F1-score: 0.58

Weighted Precision: 0.76
Weighted Recall: 0.81
Weighted F1-score: 0.78

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       0.94      1.00      0.97    105019
                       Bot       0.95      0.96      0.96       280
                      DDoS       0.88      0.97      0.93     19271
             DoS GoldenEye       0.98      0.96      0.97      1542
                  DoS Hulk       0.00      0.00      0.00     34547
          DoS Slowhttptest       0.29      0.99      0.44       828
             DoS slowloris       0.24      0.98      0.38       834
               FTP-Patator       0.99      1.00      0.99      1178
                Heartbleed       0.05      1.00      0.09         2
              Infiltration       0.00      0.00      0.00         1
             

In [ ]:
ensemble = np.zeros((y_test_origin.shape[0], 15))

ensemble = ensemble+random_pred_filter*10+dnn_sd_pred_filter + y_pred_dnn_mix + dnn_n_pred_filter+tree_pred_filter*10

In [ ]:
display_metrics(y_test_origin, np.argmax(ensemble, axis = 1) , labels_d)


Accuracy: 0.81

Micro Precision: 0.81
Micro Recall: 0.81
Micro F1-score: 0.81

Macro Precision: 0.56
Macro Recall: 0.78
Macro F1-score: 0.58

Weighted Precision: 0.76
Weighted Recall: 0.81
Weighted F1-score: 0.78

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       0.94      1.00      0.97    105019
                       Bot       0.95      0.96      0.96       280
                      DDoS       0.88      0.97      0.93     19271
             DoS GoldenEye       0.98      0.96      0.97      1542
                  DoS Hulk       0.00      0.00      0.00     34547
          DoS Slowhttptest       0.29      0.99      0.44       828
             DoS slowloris       0.24      0.98      0.38       834
               FTP-Patator       0.99      1.00      0.99      1178
                Heartbleed       0.05      1.00      0.09         2
              Infiltration       0.00      0.00      0.00         1
             

## Conclusion

 